In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
df=pd.read_csv('Real_Combine.csv')

In [3]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
X = df.iloc[:,:-1] #independent
y = df.iloc[:,-1] #dependent

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 25)): # hidden layers can range between 2 to 24
        model.add(layers.Dense(units = hp.Int('Units_' +str(i), min_value = 32, max_value = 256, step = 32), # min no of neuron in a HL is 32 and max is 256
                                        activation = 'relu')) # AF in HL is relu
        model.add(layers.Dense(1, activation = 'linear')) # this is OP layer with AF as linear
        model.compile(
            optimizer = keras.optimizers.Adam(hp.Choice('Learning_Rate', [1e-2, 1e-3, 1e-4])), #Learning rate is 0.001, 0.0001 and 0.00001
            loss = 'mean_absolute_error',
            metrics = ['mean_absolute_error'])
        return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective = "val_mean_absolute_error", #this is for regression. For classification "val_accuracy"
    max_trials = 5,
    executions_per_trial =3, #totally runs 5 X 3 times = 15 times
    directory = 'my_dir',
    project_name = 'hyperparameter_tuning_using_Keras_Tuner')

INFO:tensorflow:Reloading Oracle from existing project my_dir\hyperparameter_tuning_using_Keras_Tuner\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\hyperparameter_tuning_using_Keras_Tuner\tuner0.json


In [7]:
tuner.search_space_summary()

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [9]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary() #Score or MAE should be less to have better model